In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 7.2701 - accuracy: 0.6090 - precision: 0.5962 - recall: 0.5140 - auc: 0.6324
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 0.2230 - accuracy: 0.9288 - precision: 0.9172 - recall: 0.9322 - auc: 0.9747
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0432 - accuracy: 0.9880 - precision: 0.9952 - recall: 0.9790 - auc: 0.9992
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0050 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000
Epoch 5/10
29/29 [==============================] - 1s 17ms/step - loss: 0.0017 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 8.5548e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000
Epoch 7/10
29/29 [==============================] - 1s 18ms/step - loss: 6.8928e-04 - accuracy: 

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3,input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 24ms/step - loss: 36.0092 - accuracy: 0.5115 - precision_5: 0.4710 - recall_5: 0.3411 - auc_5: 0.5174
Epoch 2/10
29/29 [==============================] - 1s 24ms/step - loss: 1.9037 - accuracy: 0.6243 - precision_5: 0.6087 - recall_5: 0.5561 - auc_5: 0.6721
Epoch 3/10
29/29 [==============================] - 1s 23ms/step - loss: 0.5454 - accuracy: 0.7996 - precision_5: 0.7765 - recall_5: 0.8037 - auc_5: 0.8784
Epoch 4/10
29/29 [==============================] - 1s 23ms/step - loss: 0.3318 - accuracy: 0.8664 - precision_5: 0.8525 - recall_5: 0.8645 - auc_5: 0.9394
Epoch 5/10
29/29 [==============================] - 1s 23ms/step - loss: 0.2171 - accuracy: 0.9189 - precision_5: 0.9136 - recall_5: 0.9136 - auc_5: 0.9709
Epoch 6/10
29/29 [==============================] - 1s 24ms/step - loss: 0.1632 - accuracy: 0.9430 - precision_5: 0.9372 - recall_5: 0.9416 - auc_5: 0.9855
Epoch 7/10
29/29 [==============================] - 1s 23ms/ste

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 17ms/step - loss: 6.2823 - accuracy: 0.5674 - precision_10: 0.5465 - recall_10: 0.4533 - auc_10: 0.5939
Epoch 2/10
29/29 [==============================] - 0s 17ms/step - loss: 0.4230 - accuracy: 0.8028 - precision_10: 0.7731 - recall_10: 0.8201 - auc_10: 0.8946
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 0.1632 - accuracy: 0.9540 - precision_10: 0.9595 - recall_10: 0.9416 - auc_10: 0.9836
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0367 - accuracy: 0.9880 - precision_10: 0.9860 - recall_10: 0.9883 - auc_10: 0.9993
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0405 - accuracy: 0.9880 - precision_10: 0.9860 - recall_10: 0.9883 - auc_10: 0.9983
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.0162 - accuracy: 0.9967 - precision_10: 0.9977 - recall_10: 0.9953 - auc_10: 1.0000
Epoch 7/10
29/29 [============================